In [12]:
from sage.interfaces.gap import gap
import random
import time

def p_part_of_number(p, num):
    """
    Returns the p-part of num
    """
    # Convert p, num to int
    p, num = int(p), int(num)
    p_part = 1

    # While p-part still in num, add to p_part and divide num by p 
    while num % p == 0:
        num //= p
        p_part *= p
    return p_part


def set_maker(num):
    """ 
    Creates a set of numbers from 1 to log_2(num) based on binary representation of num
    """
    n = 1
    s = gap.Set([])

    # While num not zero, add nth number to set if num is odd, and divide num by 2
    while num != 0:
        if num % 2 == 1:
            s.AddSet(n)
        num = num // 2
        n += 1
    #print("hi im your set ", str(s))
    return s
    
    
def partition_finder(deg, ind):
    """
    For primitive permutation group defined by deg, ind, attempts to find a partition that is not half of the set
    with a stabilizer that is p' for every prime divisor p >= 5 dividing the order of the group
    
    Returns verification which tracks which primes it successfully found a partition for,
    and good_partition which returns each successful partition
    """
    
    # Generate list of numbers based on degree of group
    numbers = gap.Set([1..deg])

    # Create group and find its order
    G = gap.PrimitiveGroup(deg,ind)
    order = int(G.Order())

    # Find prime divisors of group >= 5
    prime_divisors = [p for p in map(int, gap.PrimeDivisors(order)) if p >= 5]
    print("prime_divisors is :" + str(prime_divisors))
    verification = {p: False for p in prime_divisors}
    good_partitions = []

    n = 0
    # Finds partitions according to specification (randomly generates partitions using partition finder)
    for i in range(2**(deg-1)):
        random_index = random.randint(1, 2**(deg))
        #print(random_index)
        partition = set_maker(random_index)
        partition_size = gap.Size(partition)
        if partition_size == deg//2 or partition_size == 0 or partition_size == deg :
            continue
        stab1 = gap.Stabilizer(G, partition, "OnSets")
        size = gap.Size(stab1)
        #print("I'm stab1 ", stab1)
        #print("stab1 is size ", size)
        n += 1
        for p in prime_divisors:
            if p_part_of_number(p, size) == 1:
                verification[p] = True
                good_partitions.append(partition)
                prime_divisors.remove(p)
                break
        #print(len(prime_divisors))
        if len(prime_divisors) == 0:
            return verification, good_partitions

    #Return results
    return verification, good_partitions



def half_size_checker(partitions, order):
    """ 
    Verifies if no partitions in a set of partitions are half the size of order
    """
    order = int(order)
    for orbit in orbits:
        if len(orbit[1]) == order/2:
            print(orbit)
            return false
    return true

def num_group_deg(deg):
    """
    Returns out the number of primitive permutation groups of degree deg
    """
    n = 1
    while true:
        try: 
            gap.PrimitiveGroup(deg,n)
            n += 1
        except:
            n -= 1
            print("I did num_group_deg")
            return n

def multi_tester(deg, max_ind):
    """
    Given degree deg and the maximum index for primitive permutation groups of degree deg, 
    tests and prints results for all groups of degree deg
    """
    for i in range(1, max_ind + 1):
        verification_results, good_partitions = partition_finder(deg, i)
        print("Verification of all primes:", verification_results)
        print("Detailed results for each prime:", verification_results)
        print(good_partitions)

def single_tester(deg, ind):
    """
    For primitive permutation group of degree deg and index ind, tests and prints results from partition_finder
    """
    verification_results, good_partitions = partition_finder(deg, ind)
    print("Verification of all primes:", verification_results)
    print("Detailed results for each prime:", verification_results)
    print(good_partitions)

In [13]:
single_tester(24, 2)

prime_divisors is :[11, 23]
Verification of all primes: {11: True, 23: True}
Detailed results for each prime: {11: True, 23: True}
[[ 1, 3, 4, 7, 8, 9, 10, 13, 15, 17, 18, 19, 21, 22, 23 ], [ 1, 2, 5, 6, 7, 8, 11, 14, 15, 16, 17, 18, 19, 22, 23, 24 ]]


In [15]:
n = 10
multi_tester(n, num_group_deg(n))

I did num_group_deg
prime_divisors is :[5]
Verification of all primes: {5: True}
Detailed results for each prime: {5: True}
[[ 2, 4, 8, 10 ]]
prime_divisors is :[5]
Verification of all primes: {5: True}
Detailed results for each prime: {5: True}
[[ 2, 4, 7 ]]
prime_divisors is :[5]
Verification of all primes: {5: True}
Detailed results for each prime: {5: True}
[[ 2, 3, 5, 6, 7, 8, 9, 10 ]]
prime_divisors is :[5]
Verification of all primes: {5: True}
Detailed results for each prime: {5: True}
[[ 1, 2, 5, 8, 9, 10 ]]
prime_divisors is :[5]
Verification of all primes: {5: True}
Detailed results for each prime: {5: True}
[[ 1, 5, 6, 7 ]]
prime_divisors is :[5]
Verification of all primes: {5: True}
Detailed results for each prime: {5: True}
[[ 1, 3, 4, 7, 9, 10 ]]
prime_divisors is :[5]
Verification of all primes: {5: True}
Detailed results for each prime: {5: True}
[[ 2, 3, 5, 6 ]]
prime_divisors is :[5, 7]
Verification of all primes: {5: False, 7: True}
Detailed results for each prime: {